In [1]:
# imports
import graphlab as gl
import numpy as np
import pandas as pd

In [2]:
# load data
train_sf = gl.SFrame('TRAIN.csv')
test_sf = gl.SFrame('TEST.csv')

train_sf.column_names

This non-commercial license of GraphLab Create for academic use is assigned to johnthas@yahoo.com and will expire on October 31, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1517566295.log


Finished parsing file /Users/robert/Documents/Kaggle/DSN Telecoms Churn 2018/TRAIN.csv

Parsing completed. Parsed 100 lines in 0.033085 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,float,float,float,float,float,int,int,int,int,str,str,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Parsing completed. Parsed 100 lines in 0.022134 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,float,float,float,float,float,int,int,int,int,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/robert/Documents/Kaggle/DSN Telecoms Churn 2018/TEST.csv

Parsing completed. Parsed 600 lines in 0.015013 secs.

<bound method SFrame.column_names of Columns:
	Customer ID	str
	network_age	int
	Customer tenure in month	float
	Total Spend in Months 1 and 2 of 2017	float
	Total SMS Spend	float
	Total Data Spend	float
	Total Data Consumption	float
	Total Unique Calls	int
	Total Onnet spend	int
	Total Offnet spend	int
	Total Call centre complaint calls	int
	Network type subscription in Month 1	str
	Network type subscription in Month 2	str
	Most Loved Competitor network in in Month 1	str
	Most Loved Competitor network in in Month 2	str
	Churn Status	int

Rows: 1401

Data:
+-------------+-------------+--------------------------+-------------------------------+
| Customer ID | network_age | Customer tenure in month | Total Spend in Months 1 an... |
+-------------+-------------+--------------------------+-------------------------------+
|   ADF0039   |     123     |           4.1            |             76.714            |
|   ADF0041   |     1316    |          43.87           |            98.8904      

### churn prediction requires a time series. we don't have this, so consider linear regression.

In [3]:
train_sf = train_sf.fillna('Network type subscription in Month 1', '3G')
train_sf = train_sf.fillna('Network type subscription in Month 2', '3G')

train_sf = train_sf.fillna('Most Loved Competitor network in in Month 1', 'Uxaa')
train_sf = train_sf.fillna('Most Loved Competitor network in in Month 2', 'Uxaa')

train_sf = train_sf.dropna(columns=None, how='any')

In [4]:
# Make a train-test split
train_data, test_data = train_sf.random_split(0.8)

In [5]:
feature_columns = ['network_age', 'Customer tenure in month', 'Total Spend in Months 1 and 2 of 2017', 'Total SMS Spend', 'Total Data Spend', 'Total Data Consumption', 'Total Unique Calls', 'Total Onnet spend', 'Total Offnet spend', 'Total Call centre complaint calls', 'Network type subscription in Month 1', 'Network type subscription in Month 2', 'Most Loved Competitor network in in Month 1', 'Most Loved Competitor network in in Month 2']
# Create a model.
model = gl.linear_regression.create(train_data, target='Churn Status',
                                    features = feature_columns)

# Save predictions to an SFrame (class and corresponding class-probabilities)
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
results = model.evaluate(test_data)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Linear regression:

--------------------------------------------------------

Number of examples          : 1076

Number of features          : 14

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 1.013400     | 1.216641           | 0.893124             | 0.451509      | 0.502875        |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

SUCCESS: Optimal solution found.

In [6]:
results

{'max_error': 1.0496128420331277, 'rmse': 0.45760978727521423}

In [7]:
model['coefficients']

name,index,value,stderr
(intercept),None,0.651006007377,nan
network_age,None,-1.77753635142e-05,nan
Customer tenure in month,None,-0.000498419722712,nan
Total Spend in Months 1and 2 of 2017 ...,None,-6.58784746371e-05,nan
Total SMS Spend,None,0.000739111898387,nan
Total Data Spend,None,0.000139536702939,nan
Total Data Consumption,None,-9.68121121565e-09,nan
Total Unique Calls,None,-8.81336719119e-05,nan
Total Onnet spend,None,-1.8368919953e-07,nan
Total Offnet spend,None,3.04927288251e-06,nan


In [8]:
predictions = model.predict(test_sf)

In [9]:
type(predictions)

graphlab.data_structures.sarray.SArray

In [10]:
type(test_sf)

graphlab.data_structures.sframe.SFrame

In [11]:
test_sf['predictions'] = predictions

In [12]:
test_sf.head()

Customer ID,network_age,Customer tenure in month,Total Spend in Months 1and 2 of 2017 ...,Total SMS Spend,Total Data Spend
ADF0001,1012,33.73333333,1560.1156,14.04,226.3
ADF0002,137,4.566666667,1705.8368,73.63,5.0
ADF0003,202,6.733333333,1720.3008,4.36,245.0
ADF0004,165,5.5,1732.28,0.0,12.5
ADF0005,759,25.3,1873.06,69.57,412.5
ADF0006,129,4.3,1877.26,46.11,343.75
ADF0007,148,4.933333333,2388.8208,17.99,1.25
ADF0008,1144,38.13333333,2702.35,47.84,48.75
ADF0009,137,4.566666667,2782.5572,179.52,315.0
ADF0010,1456,48.53333333,2934.7028,200.37,108.75


In [13]:
test_sf['Churn Status'] = test_sf['predictions'].apply(lambda x: int(x > 0.5))

In [14]:
test_sf.head()

Customer ID,network_age,Customer tenure in month,Total Spend in Months 1and 2 of 2017 ...,Total SMS Spend,Total Data Spend
ADF0001,1012,33.73333333,1560.1156,14.04,226.3
ADF0002,137,4.566666667,1705.8368,73.63,5.0
ADF0003,202,6.733333333,1720.3008,4.36,245.0
ADF0004,165,5.5,1732.28,0.0,12.5
ADF0005,759,25.3,1873.06,69.57,412.5
ADF0006,129,4.3,1877.26,46.11,343.75
ADF0007,148,4.933333333,2388.8208,17.99,1.25
ADF0008,1144,38.13333333,2702.35,47.84,48.75
ADF0009,137,4.566666667,2782.5572,179.52,315.0
ADF0010,1456,48.53333333,2934.7028,200.37,108.75


In [15]:
churn_sf = test_sf[['Customer ID', 'Churn Status']]

In [16]:
churn_sf.export_csv('predictions/turi_linear_regression.csv')

### Submission Accuracy: 0.69444

In [17]:
# investigate coefficients further
# model['coefficients'].show()

## Play around with features

In [18]:
def polynomial_sframe(sf, degree, target):
    nsf = gl.SFrame()
    temp = sf[target]
    
    
    sf.remove_column(target)
    columns = sf.column_names()
    
    for column in columns:
        sa = sf[[column]]
        
        for d in range(1, degree + 1):
            nsf.add_column(sa.apply(lambda x: x[column] ** d))
    nsf.add_column(temp, target)
    return nsf

In [19]:
def polynomial_sframe_test(sf, degree):
    nsf = gl.SFrame()
    
    columns = sf.column_names()
    
    for column in columns:
        sa = sf[[column]]
        
        for d in range(1, degree + 1):
            nsf.add_column(sa.apply(lambda x: x[column] ** d))
    return nsf

In [20]:
c = ['Churn Status','network_age','Customer tenure in month','Total Spend in Months 1 and 2 of 2017','Total SMS Spend','Total Data Spend','Total Data Consumption','Total Unique Calls','Total Onnet spend','Total Offnet spend','Total Call centre complaint calls']

In [21]:
new_train = polynomial_sframe(train_sf[c], 6, 'Churn Status')
new_test = polynomial_sframe_test(test_sf[c], 6)
new_train.head()

X1,X2,X3,X4,X5,X6,X7,X8
123,15129,1860867,228886641,28153056843,3.46282599169e+12,4.1,16.81
1316,1731856,2279122496,2999325204736,3947111969432576,5.19439935177e+18,43.87,1924.5769
2385,5688225,13566416625,32355903650625,77168830206740625,1.84047660043e+20,79.5,6320.25
1614,2604996,4204463544,6786004160016,10952610714265824,1.76775136928e+19,53.8,2894.44
2175,4730625,10289109375,22378812890625,48673918037109375,1.05865771731e+20,72.5,5256.25
158,24964,3944312,623201296,98465804768,1.55575971533e+13,5.27,27.7729
110,12100,1331000,146410000,16105100000,1.771561e+12,3.67,13.4689
558,311364,173741112,96947540496,54096727596768,3.0185973999e+16,18.6,345.96
115,13225,1520875,174900625,20113571875,2.31306076562e+12,3.83,14.6689
1868,3489424,6518244032,12176079851776,22744917163117568,4.24875052607e+19,62.27,3877.5529


In [22]:
# Make a train-test split
train_data, test_data = new_train.random_split(0.8)

In [23]:
penalties = []
l2_range = np.logspace(1, 7, num=13)
for l2_penalty in l2_range:
    #create a model
    model = gl.linear_regression.create(train_data, target='Churn Status', l2_penalty=l2_penalty)
    #valuate model
    results = model.evaluate(test_data)
    penalties.append( (l2_penalty, results) )

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Linear regression:

--------------------------------------------------------

Number of examples          : 1068

Number of features          : 60

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.061071     | 0.501091           | 0.500967             | 0.499985      | 0.500101        |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

SUCCESS: Optimal solution found.

In [24]:
penalties

[(10.0, {'max_error': 56.06998629052236, 'rmse': 3.505755951806999}),
 (31.622776601683793,
  {'max_error': 49.96692591125324, 'rmse': 3.132717670616174}),
 (100.0, {'max_error': 26.708775291423493, 'rmse': 1.7288003126410993}),
 (316.22776601683796,
  {'max_error': 30.24014540556942, 'rmse': 1.9341800038341301}),
 (1000.0, {'max_error': 10.432073425399201, 'rmse': 0.8089678919006067}),
 (3162.2776601683795,
  {'max_error': 10.993298277834075, 'rmse': 0.838631547819895}),
 (10000.0, {'max_error': 14.756197338365894, 'rmse': 1.04242568783367}),
 (31622.776601683792,
  {'max_error': 3.5183464815866623, 'rmse': 0.5431374378259664}),
 (100000.0, {'max_error': 1.0138552214750822, 'rmse': 0.5024461304165243}),
 (316227.76601683791,
  {'max_error': 0.5102290433215652, 'rmse': 0.498990033555177}),
 (1000000.0, {'max_error': 0.5062236594405898, 'rmse': 0.49952286338584234}),
 (3162277.6601683795,
  {'max_error': 0.507721376629376, 'rmse': 0.49996610777768036}),
 (10000000.0, {'max_error': 0.501

In [25]:
# Save predictions to an SFrame (class and corresponding class-probabilities)
#predictions = model.predict(new_test)

In [26]:
#model['coefficients'].print_rows(num_rows=61)